## Statistics 504 Homework #1 <br>
Fall 2015 <br>
Erich Seamon <br>
University of Idaho <br>
erichs@uidaho.edu <br>
http://github.com/erichseamon.com/stat504-python <br>

## Exercise 1
### Write a program that reads a file, breaks each line into words, strips whitespace and punctuation from the words, and converts them to lowercase.

#### Solution:  

Below is a set of defined functions that accomplish Exercise 1.

#### Functions used as part of Exercise 1: 

-process_file - a function that calls the filename that will be set in main - defines a binary skip_header argument, and then processes each line in the datafile. <br><br>
-skip_gutenberg_header - a function that skips the header of a file <br><br>
-process_line - goes thru each line and breaks each word apart, strips out puncuation<br>

These three functions complete Exercise 1.

In [136]:
import operator
import string
import random
import urllib2
from string import digits
import re


### Function definitions

##### Process_file - a function that creates the histogram/list of words after other functions below are run to prepare the data.

In [137]:
def process_file(filename, skip_header):
    """Makes a histogram that contains the words from a file.

    filename: string
    skip_header: boolean, whether to skip the Gutenberg header
   
    Returns: map from each word to the number of times it appears.
    """
    hist = {}
    fp = file(filename)

    if skip_header:
        skip_gutenberg_header(fp)

    for line in fp:
        process_line(line, hist)
    return hist

###  Function that skips the header

In [138]:
def skip_gutenberg_header(fp):
    """Reads from fp until it finds the line that ends the header.

    fp: open file object
    """
    for line in fp:
        if line.startswith('*END*THE SMALL PRINT!'):
            break

### Process_line function, that splits words and strips unnecessary information

In [139]:
def process_line(line, hist):
    """Adds the words in the line to the histogram.
    Modifies hist.
    line: string
    hist: histogram (map from word to frequency)
    """
    # replace hyphens with spaces before splitting
    line = line.replace('-', ' ')
    #replace the colon in advance between chapter and verse
    line = line.replace(':', '')

    for word in line.split():
        # remove punctuation and convert to lowercase

        word = word.strip(string.whitespace)
        word = word.strip()
        word = word.strip(string.punctuation)
        word = word.lower()
        word = word.replace("'", "") #added to remove extraneous leftover apostrophes
        word = re.sub("\S*\d\S*", "", word).strip()
        word = word.strip("" + string.whitespace)

        # update the histogram
        hist[word] = hist.get(word, 0) + 1

In [140]:
def most_common(hist, trim):
    """Makes a list of the key-value pairs from a histogram and
    sorts them in descending order by frequency.
    
    threshold = the cutoff threshold for frequency occurances"""
    
    t = []
    for key, value in hist.items(): 
        if value >= trim:  #--added by es to enable k-fold list
            t.append((value, key))
    t.sort()
    t.reverse() 
    
    return t



## Exercise 2
### Go to Project Gutenberg (http://gutenberg.org) and download The King James Bible available at http://www.gutenberg.org/cache/epub/10/pg10.txt in plain text format. 

Modify your program from the previous exercise to read The King James Bible you downloaded, skip over the header information at the beginning of the file, and process the rest of the words as before. 

Then modify the program to count the total number of words in the Old Testament, and print out the number of times each word is used.  alongside the word if the word is used more than k times (where k as a user input parameter) in decreasing order. Print out the output when k=100.  (list of all the words – with their count beside them – with a cutoff of 100 (k).  K is input).  

Repeat the above for the New Testament. 


#### Solution:  

Building upon the solution provided in Exercise 1 - below is:

-a set of defined functions that accomplish Exercise 2, as well as: <br>
-a __main__ code section that calls all functions and sets the data source. <br>

The output below counts the total number of words in the Old Testament, with a frequency of 100 cutoff.

#### Functions used as part of Exercise 2: <br>

-most_common - determines the most frequent words.  This function is where the k threshold was set. <br>
-print_most_common - prints the most common words determined from most_common <br>
-subract - Removes any words that we might not want to include - such as prepositions, pronouns, etc. <br>
-total_words - Does a count of total words <br>

### Loading King James File
##### Loading the king james file from a url.  This is a url location online where i dumped the kingjames.txt file.  I did this because the web site where the file resides online was blocking my Python access - i believe the site has some preventative measures on code repeatedly accessing data on the site.  So my remedy was just to move it to another server location i have access to and then download it from there. 

In [141]:
book = urllib2.urlopen('https://www.reacchpna.org/files/7614/4236/8760/kingjames.txt').read()

### Print_most_common function - prints the words that occur at least 100 times - or the number of times specified in the main section, where this function is called.

In [142]:
def print_most_common(hist, num=10):
    """Prints words that occur at least K times in a histgram and their frequencies.
    in this instance, 100 is used.
    
    hist: histogram (map from word to frequency
    threshold: the lowest frequency of words to show.
    """
    
    t = most_common(hist)
    print 'The most common words are:'
    for freq, word in t[:num]:
        print word, '\t', freq

In [143]:
def subtract(d1, d2):
    """Returns a dictionary with all keys that appear in d1 but not d2.

    d1, d2: dictionaries
    """
    res = {}
    for key in d1:
        if key not in d2:
            res[key] = None
    return res

In [144]:
def total_words(hist):
    """Returns the total of the frequencies in a histogram."""
    return sum(hist.values())

In [145]:
def different_words(hist):
    hist.pop("", None) #-added to remove white space oddness

    """Returns the number of different words in a histogram."""
    return len(hist)

##### Run all the functions for the Old Testament

In [146]:
if __name__ == '__main__':
    
    f = book.split('\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n')[1]
    lines = f.rpartition('The New Testament of the King James Bible')
    NewT = f.split('***')[1]

    OldTestament = lines[0]
    g = lines[2]
    NT = NewT.rpartition('End of the Project Gutenberg EBook of The King James Bible') 
    NewTestament = NT[0]

    # sends output to a variable

    i = open('OldTestament.txt','w')
    print >>i, OldTestament
    hist = process_file('OldTestament.txt', skip_header=False)
    hist.pop("", None) #-added to remove white space oddness
    print 'Total number of words:', sum(hist.values())
    print 'Number of different words:', different_words(hist)

    t = most_common(hist, 100)
    print 'The words that occur in the OLD TESTAMENT at least 100 times, in decending order, are:'
    for freq, word in t:
        print word, '\t', freq
        #print word, freq

    #print "\n\nHere are some random words from the book"
    #for i in range(100):
        #print random_word(hist),
        


Total number of words: 609446
Number of different words: 10754
The words that occur in the OLD TESTAMENT at least 100 times, in decending order, are:
the 	52992
and 	40975
of 	28548
to 	10207
in 	9767
that 	9152
shall 	8423
he 	7347
lord 	7119
his 	7036
for 	6689
i 	6669
unto 	6551
a 	6268
they 	5411
be 	5296
it 	4852
is 	4820
them 	4730
with 	4643
thou 	4617
not 	4533
all 	4490
him 	4193
thy 	4184
my 	3749
their 	3445
thee 	3344
will 	3281
me 	3253
was 	3172
god 	3089
said 	2938
from 	2910
which 	2860
have 	2820
israel 	2490
upon 	2485
as 	2480
ye 	2424
but 	2323
out 	2267
king 	2171
were 	1989
are 	1987
up 	1971
this 	1968
son 	1948
people 	1908
when 	1862
house 	1818
by 	1815
man 	1744
there 	1732
hath 	1707
her 	1690
land 	1668
children 	1645
came 	1587
then 	1581
shalt 	1509
before 	1503
against 	1457
on 	1437
day 	1418
an 	1385
come 	1384
had 	1383
you 	1373
into 	1371
one 	1360
your 	1323
so 	1314
hand 	1308
men 	1267
go 	1240
at 	1180
also 	1170
let 	1169
made 	1119
even 	1118


### Exercise 3 - Below we run the subtract function to eliminate any words in our words list from the OLD TESTAMENT.  This list happens to be a list of prepositions, pronouns, conjunctions, and articles for removal.


In [147]:
    words = process_file('/git/stat504-python/STAT504-HW1/HW1-wordlist.txt', skip_header=False)

    diff = subtract_words(hist, words)
    dc = most_common(diff, 100)

    print "Total words are in the OLD TESTAMENT that are not in the wordlist:", sum(diff.values())
    print "The words in the OLD TESTAMENT that are not in the wordlist, that occur at least 100 times, descending order:"
    for freq, word in dc:
        print word, '\t', freq
    
    #for word in diff.keys():
        #print word, '\t', 

Total words are in the OLD TESTAMENT that are not in the wordlist: 245211
The words in the OLD TESTAMENT that are not in the wordlist, that occur at least 100 times, descending order:
lord 	7119
god 	3089
said 	2938
israel 	2490
king 	2171
son 	1948
people 	1908
house 	1818
man 	1744
land 	1668
children 	1645
came 	1587
day 	1418
come 	1384
one 	1360
hand 	1308
men 	1267
go 	1240
made 	1119
behold 	1104
sons 	1084
went 	1062
saying 	990
saith 	955
david 	953
do 	952
now 	949
make 	910
judah 	811
earth 	795
moses 	772
name 	755
put 	754
brought 	730
heart 	725
city 	716
take 	713
offering 	713
pass 	712
great 	708
days 	708
away 	704
two 	698
jerusalem 	667
according 	666
bring 	641
say 	628
father 	627
fathers 	620
set 	611
place 	611
kings 	588
egypt 	587
took 	586
hundred 	556
way 	527
eat 	520
word 	491
sent 	489
years 	483
evil 	481
good 	472
thousand 	466
fire 	466
know 	464
time 	453
spake 	451
words 	450
again 	445
things 	433
holy 	430
cities 	429
eyes 	428
priest 	427
done 	42

## Repeating Exercise 1,2, and 3 for the New Testament

### Exercise 1 and 2 - Given the previously described functions, run all of them for the NEW TESTAMENT

In [148]:
if __name__ == '__main__':
    
    f = book.split('\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n')[1]
    lines = f.rpartition('The New Testament of the King James Bible')
    NewT = f.split('***')[1]

    OldTestament = lines[0]
    g = lines[2]
    NT = NewT.rpartition('End of the Project Gutenberg EBook of The King James Bible') 
    NewTestament = NT[0]

    # sends output to a variable

    h = open('NewTestament.txt','w')
    print >>h, NewTestament

    hist = process_file('NewTestament.txt', skip_header=False)
    hist.pop("", None) #-added to remove white space oddness
    print 'Total number of words:', total_words(hist)
    print 'Number of different words:', different_words(hist)
    
    t = most_common(hist, 100)
    
    print 'The words that occur in the NEW TESTAMENT at least 100 times, in decending order, are:'
    for freq, word in t:
        print word, '\t', freq
        #print word, freq

    #print "\n\nHere are some random words from the book"
    #for i in range(100):
        #print random_word(hist),

Total number of words: 180603
Number of different words: 6002
The words that occur in the NEW TESTAMENT at least 100 times, in decending order, are:
the 	11031
and 	10721
of 	6122
that 	3760
to 	3373
he 	3072
in 	2900
him 	2466
unto 	2446
for 	2281
i 	2185
is 	2169
not 	2063
they 	1965
a 	1909
be 	1717
them 	1700
but 	1669
ye 	1559
which 	1553
his 	1437
shall 	1415
with 	1369
god 	1356
was 	1350
it 	1277
you 	1244
all 	1130
have 	1084
said 	1061
as 	1040
jesus 	983
when 	972
are 	963
we 	869
man 	869
thou 	857
me 	843
this 	817
by 	810
were 	783
from 	732
things 	729
lord 	711
into 	644
had 	643
my 	619
one 	607
if 	602
also 	599
then 	588
come 	587
on 	574
there 	567
hath 	557
will 	555
christ 	555
out 	508
came 	506
us 	489
their 	487
thee 	483
no 	470
your 	455
saying 	455
say 	428
son 	422
thy 	416
do 	416
up 	409
what 	408
now 	407
these 	392
at 	390
men 	386
who 	378
so 	375
should 	366
our 	366
therefore 	356
father 	352
let 	342
went 	338
or 	324
day 	316
an 	314
saith 	307
who

### Exercise 3 - Below we run the subtract function to eliminate any words in our words list from the NEW TESTAMENT.  This list happens to be a list of prepositions, pronouns, conjunctions, and articles for removal.


In [149]:
    words = process_file('/git/stat504-python/STAT504-HW1/HW1-wordlist.txt', skip_header=False)

    diff = subtract_words(hist, words)
    dc = most_common(diff, 100)

    print "Total words are in the NEW TESTAMENT that are not in the wordlist:", sum(diff.values())
    print "The words in the NEW TESTAMENT that are not in the wordlist, that occur at least 100 times, descending order:"
    for freq, word in dc:
        print word, '\t', freq
    
    #for word in diff.keys():
        #print word, '\t', 
        

Total words are in the NEW TESTAMENT that are not in the wordlist: 72611
The words in the NEW TESTAMENT that are not in the wordlist, that occur at least 100 times, descending order:
god 	1356
said 	1061
jesus 	983
man 	869
things 	729
lord 	711
one 	607
come 	587
christ 	555
came 	506
saying 	455
say 	428
son 	422
do 	416
now 	407
men 	386
father 	352
went 	338
day 	316
saith 	307
know 	299
made 	286
spirit 	261
heaven 	255
great 	254
go 	252
heard 	248
good 	248
faith 	245
disciples 	243
world 	241
might 	239
people 	231
brethren 	231
called 	229
again 	227
law 	223
behold 	222
saw 	221
another 	214
away 	211
see 	208
word 	206
own 	206
house 	206
answered 	203
sent 	196
earth 	192
being 	190
dead 	187
life 	186
holy 	181
love 	179
glory 	177
jews 	175
same 	174
name 	173
time 	170
paul 	170
days 	169
first 	167
cast 	167
took 	164
take 	161
speak 	161
peter 	160
kingdom 	158
hand 	158
put 	157
children 	157
having 	155
power 	152
city 	152
those 	151
other 	150
flesh 	149
some 	148


NameError: name 'NewTestament' is not defined